In [1]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy import stats
import statsmodels.api as sm



c:\Users\buckl\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
c:\Users\buckl\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
ffm = pd.read_csv('F-F_Research_Data_Factors.CSV')
ffm['Date'] = pd.to_datetime(ffm['Date'],format='%Y%m')
ffm.set_index('Date', inplace=True)

In [3]:
ffm

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-01,2.96,-2.56,-2.43,0.22
1926-08-01,2.64,-1.17,3.82,0.25
1926-09-01,0.36,-1.40,0.13,0.23
1926-10-01,-3.24,-0.09,0.70,0.32
1926-11-01,2.53,-0.10,-0.51,0.31
...,...,...,...,...
2023-09-01,-5.24,-2.51,1.52,0.43
2023-10-01,-3.19,-3.87,0.19,0.47
2023-11-01,8.84,-0.02,1.64,0.44


In [4]:
ticker = 'AAPL'

In [5]:
ticker_data = pd.read_csv(f'..\Life On The Edge Fund\{ticker}.csv')
ticker_data['Date'] = pd.to_datetime(ticker_data['Date'])
ticker_data.set_index('Date', inplace=True)



In [6]:
start_date = max([ticker_data.index[0],pd.to_datetime('2006-01-01')])
start_date

Timestamp('2006-01-01 00:00:00')

In [7]:
ticker_price = ticker_data[ticker_data.index>=start_date]['Adj Close']
ticker_return = 100 * np.log(ticker_price/ticker_price.shift(1))
mkt_minus_rf= ffm[ffm.index>start_date]['Mkt-RF']
ticker_return_minus_rf = ticker_return - ffm[ffm.index >= start_date]['RF']


In [8]:
avg_mkt_minus_rf_return = mkt_minus_rf.mean()/100
avg_smb_return = ffm[ffm.index>=start_date]['SMB'].mean()/100
avg_hml_return = ffm[ffm.index>=start_date]['HML'].mean()/100
avg_rf_return = ffm[ffm.index>=start_date]['RF'].mean()/100
avg_smb_return

6.082949308755754e-05

In [9]:
mkt_minus_rf_sd = np.std(mkt_minus_rf)
mkt_minus_rf_var = np.var(mkt_minus_rf)
ticker_return_rf_mkt_rf_corr = np.corrcoef(ticker_return_minus_rf.dropna(),mkt_minus_rf.dropna())[0,1]
ticker_return_rf_mkt_rf_cov = np.cov(ticker_return_minus_rf.dropna(),mkt_minus_rf.dropna())[0,1]


CAPM Beta

In [10]:
y = np.array(ticker_return_minus_rf.dropna())
X = np.array(mkt_minus_rf)
X = sm.add_constant(X)
model = sm.OLS(y,X)
results = model.fit()
intercept, beta = results.params 

print(f"Beta {beta}")



Beta 1.183486180159068


FFM Betas

In [11]:


X_1 = np.array(mkt_minus_rf)
X_2 = np.array(ffm[ffm.index>start_date]['SMB'])
X_3 = np.array(ffm[ffm.index>start_date]['HML'])
X = np.column_stack((X_1, X_2, X_3))
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
intercept, beta1, beta2, beta3 = results.params


print(f"Beta1 (Market): {beta1}, Beta2 (SMB): {beta2}, Beta3 (HML): {beta3}")


Beta1 (Market): 1.331426149163868, Beta2 (SMB): -0.34642219914966177, Beta3 (HML): -0.7150164931696684


Expected Returns (CAPM)

In [12]:
expected_monthly_return = avg_rf_return + (beta*avg_mkt_minus_rf_return)
expected_monthly_return
expected_yearly_return = np.power(1+expected_monthly_return,12) - 1
print(f'Expected monthly return: {round(expected_monthly_return*100,2)}%')
print(f'Expected yearly annualized return {round(expected_yearly_return*100,2)}%')


Expected monthly return: 1.03%
Expected yearly annualized return 13.1%


Expected Returns (FFM)

In [13]:
expected_monthly_return = avg_rf_return + beta1*avg_mkt_minus_rf_return + beta2*avg_smb_return + beta3*avg_hml_return
expected_yearly_return = np.power(1 + expected_monthly_return,12) - 1
print(f'Expected monthly return: {round(expected_monthly_return*100,2)}%')
print(f'Expected yearly annualized return {round(expected_yearly_return*100,2)}%')

Expected monthly return: 1.23%
Expected yearly annualized return 15.86%


Calculating Portfolio Betas and Expected Returns

CAPM

In [14]:
def get_ticker_beta(ticker):
    ticker_data = pd.read_csv(f'..\Out In The World Fund\{ticker}.csv')
    ticker_data['Date'] = pd.to_datetime(ticker_data['Date'])
    ticker_data.set_index('Date', inplace=True)
    start_date = max([ticker_data.index[0],pd.to_datetime('2006-01-01')])
    ticker_price = ticker_data[ticker_data.index>=start_date]['Adj Close']
    ticker_return = 100 * np.log(ticker_price/ticker_price.shift(1))
    mkt_minus_rf= ffm[ffm.index>start_date]['Mkt-RF']
    ticker_return_minus_rf = ticker_return - ffm[ffm.index >= start_date]['RF']
    y = np.array(ticker_return_minus_rf.dropna())
    X = np.array(mkt_minus_rf)
    X = sm.add_constant(X)
    model = sm.OLS(y,X)
    results = model.fit()
    intercept, beta = results.params
    return beta




In [15]:
def get_portfolio_beta(tickers,weights):
    port_beta = 0
    for ticker,weight in zip(tickers, weights):
        ticker_beta = get_ticker_beta(ticker)
        port_beta += ticker_beta*weight 
    return port_beta



In [16]:
tickers = pd.read_csv('../Out In The World Fund/weights.csv')['Ticker'].values
weights = pd.read_csv('../Out In The World Fund/weights.csv')['Weights'].values

get_portfolio_beta(tickers, weights)

1.017583474232874

FFM Betas

In [17]:
def get_ticker_betas(ticker):
    ticker_data = pd.read_csv(f'..\Out In The World Fund\{ticker}.csv')
    ticker_data['Date'] = pd.to_datetime(ticker_data['Date'])
    ticker_data.set_index('Date', inplace=True)
    start_date = max([ticker_data.index[0],pd.to_datetime('2006-01-01')])
    ticker_price = ticker_data[ticker_data.index>=start_date]['Adj Close']
    ticker_return = 100 * np.log(ticker_price/ticker_price.shift(1))
    mkt_minus_rf= ffm[ffm.index>start_date]['Mkt-RF']
    ticker_return_minus_rf = ticker_return - ffm[ffm.index >= start_date]['RF']
    y = np.array(ticker_return_minus_rf.dropna())
    X_1 = np.array(mkt_minus_rf)
    X_2 = np.array(ffm[ffm.index>start_date]['SMB'])
    X_3 = np.array(ffm[ffm.index>start_date]['HML'])
    X = np.column_stack((X_1, X_2, X_3))
    X = sm.add_constant(X)
    model = sm.OLS(y, X)
    results = model.fit()
    intercept, beta_mkt, beta_smb, beta_hml = results.params
    return (beta_mkt,beta_smb,beta_hml)



    


In [18]:
def get_portfolio_betas(tickers,weights):
    beta_mkt = 0
    beta_smb = 0
    beta_hml = 0
    for ticker,weight in zip(tickers,weights):
        betas = get_ticker_betas(ticker)
        beta_mkt+= betas[0] * weight 
        beta_smb += betas[1]*weight 
        beta_hml += betas[2]* weight 
    
    return beta_mkt, beta_smb, beta_hml

In [19]:
tickers = pd.read_csv('../Out In The World Fund/weights.csv')['Ticker'].values
weights = pd.read_csv('../Out In The World Fund/weights.csv')['Weights'].values

get_portfolio_betas(tickers, weights)

(0.970770632282987, 0.18315719109654413, -0.1873038953502487)